In [55]:
import json
from sklearn.model_selection import train_test_split
import re
from skmultilearn.model_selection import IterativeStratification
import numpy as np
import sys
from sklearn.utils import shuffle

In [56]:
def convert_escape_sequences(input_string):
    # Using regular expression to replace multiple spaces with a single space
    # input_string= re.sub(r'\s+', ' ', input_string)
    return re.sub(r'[\n\r\t\f\v]', ' ', input_string)

def bio_tagging_2(words,aspects):
    labels=['O' for i in range(len(words))]
    for i in aspects:
        s=i['term']
        start=i['from']
        end=i['to'] 
        labels[start]="B"
        for j in range(start+1,end):
            labels[j]="I"
    return labels

def bio_tagging_1(values,text):
    text=convert_escape_sequences(text)
    words=text.split(" ")
    labels=['O' for i in range(len(words))]
    curr=0
    currcount=0

    for i in values:
        i=i['value']
        start=i['start']
        end=i['end']
        s=text[start:end+1].split()
        label=i['labels'][0]
        while(curr<len(words)):
            # if ' '.join(s) in ' '.join(words[curr:curr+len(s)]):
            if currcount+len(words[curr])>=start:
                labels[curr]="B_"+label
                currcount+=len(words[curr])+1
                curr+=1
                
                while(currcount<end):
                # for j in range(len(s)-1):
                    labels[curr]='I_'+label
                    currcount+=len(words[curr])+1
                    curr+=1
                break
            currcount+=len(words[curr])+1
            curr+=1
    i=0
    while(i<len(words)):
        if words[i]=="":
            words.pop(i)
            labels.pop(i)
        i+=1
    # print(words)
    return (" ".join(words),labels)
    

## NER_JUDGEMENT

In [57]:
def file_1(input,file):
    data={}
    # print(input[0])
    for i in input:
        id=i[0]
        values=i[1]
        text=i[2]
        words,labels=bio_tagging_1(values,text)
        data[id]={"text":words,"labels":labels}
    file_path = "stratified/"+file
    with open(file_path, "w") as json_file:
        json.dump(data, json_file,indent=3)
def file_1_2(input,file):
    data={}
    for i in input:
        values=i['annotations'][0]['result']
        text=i['data']['text']
        words,labels=bio_tagging_1(values,text)
        data[i['id']]={"text":words,"labels":labels}
    file_path = "stratified/"+file
    with open(file_path, "w") as json_file:
        json.dump(data, json_file,indent=3)

### Train and Val

In [58]:

input_file="NER_TRAIN_JUDGEMENT.json"
f = open(input_file,)
input=json.load(f)
input_data=[[],[],[]]
labelset=set()
labeldict={}
for i in input:
    id=i['id']
    values=i['annotations'][0]['result']
    text=i['data']['text']
    input_data[0].append(id)
    input_data[1].append(values)
    input_data[2].append(text)
    # print(input_data[0][-1])
    for i in input_data[1][-1]:
        labelset.add(i['value']['labels'][0])
        # if i['labels'][0] not in labeldict:
        #     labeldict[i['labels'][0]]=len(labeldict)
for i in labelset:
    labeldict[i]=len(labeldict)
print(labeldict)
label_matrix=[]
for i in range(len(input_data[1])):
    label_matrix.append([0 for j in range(len(labeldict))])
for i in range(len(input_data[1])):
    for j in input_data[1][i]:
        label_matrix[i][labeldict[j['value']['labels'][0]]]=1



input_data=np.column_stack(input_data)
input_data,label_matrix=shuffle(input_data,label_matrix,random_state=42)

{'PROVISION': 0, 'ORG': 1, 'GPE': 2, 'CASE_NUMBER': 3, 'WITNESS': 4, 'RESPONDENT': 5, 'OTHER_PERSON': 6, 'STATUTE': 7, 'DATE': 8, 'COURT': 9, 'PETITIONER': 10, 'JUDGE': 11, 'PRECEDENT': 12}


In [59]:
input=input_data
label_matrix=np.array(label_matrix)

print(input.shape)
print(label_matrix.shape)

(9435, 3)
(9435, 13)


In [60]:

is1=IterativeStratification(n_splits=2, order=1, sample_distribution_per_fold=[0.85, 0.15])
for train_index, test_index in is1.split(input, label_matrix):
    X_train, X_test = input[train_index], input[test_index]
    y_train, y_test = label_matrix[train_index], label_matrix[test_index]

train=[X_train,y_train]
val=[X_test,y_test]

In [61]:

# print(X_train[0][0])
file_1(X_train,"NER_train.json")
file_1(X_test,"NER_val.json")

### Test

In [62]:
input_file="NER_TEST_JUDGEMENT.json"
f = open(input_file,)
test=json.load(f)
print(len(test))
file_1_2(test,"NER_test.json")

949


## Laptop Review

In [63]:
train=json.load(open("Laptop_Review_Train.json"))
val=json.load(open("Laptop_Review_Val.json"))
test=json.load(open("Laptop_Review_Test.json"))
def func(input,output_name):
    data={}
    count=0
    for i in input:
        words=i['words']
        aspects=i['aspects']
        text=i['raw_words']
        # print(bio_tagging_2(words,aspects))
        data[count]={"text":text,"labels":bio_tagging_2(words,aspects)}
        count+=1
        # break
    file_path = "stratified/"+output_name+".json"
    # # Write data to JSON file
    with open(file_path, "w") as json_file:
        json.dump(data, json_file,indent=3)

In [64]:
func(train,"ATE_train")
func(val,"ATE_val")
func(test,"ATE_test")